In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

Link towards reporting gsheet document:
https://docs.google.com/spreadsheets/d/1o0O9HGCUABQWF1C6uHw65veY9Axd3pATWwdG2WSpdf8/edit?usp=sharing

In [ ]:
# Parameters corresponding to gsheet file
CLASS_NBR = 6
MODE = "direct" # "direct" or "cached"
KERNEL = "linear" # "linear" or "rbf"
LR = 0.1
C = 5

In [ ]:
# Local parameters
BATCH_SIZE = 10
CNN_OUTPUT_DIR = os.path.join("..", "tmp", "cnn_output", "VGG16")
TARGET_W = 224
TRAIN_STEPS = None
MODEL_DIR_DIRECT = "../tmp/estimator_svdd_naive_direct"
MODEL_DIR_CACHED = "../tmp/estimator_svdd_naive_cached"

# Input data

In [ ]:
from data_utils import train_cnn_input_fn, test_cnn_input_fn, train_input_fn, test_input_fn

# Sanity check
train_cnn_input_fn(
    CLASS_NBR,
    CNN_OUTPUT_DIR
), test_cnn_input_fn(
    CLASS_NBR,
    CNN_OUTPUT_DIR
), train_input_fn(
    CLASS_NBR,
    TARGET_W
), test_input_fn(
    CLASS_NBR,
    TARGET_W
)

# Training

In [ ]:
from estimator_svdd_naive import OCClassifier as SVDDClassifier

if MODE == "cached":
    input_fn_train = lambda: train_cnn_input_fn(CLASS_NBR, CNN_OUTPUT_DIR).batch(BATCH_SIZE).repeat()
    input_fn_test = lambda: test_cnn_input_fn(CLASS_NBR, CNN_OUTPUT_DIR).batch(BATCH_SIZE)
    train_hooks = []
    test_hooks = train_hooks
    MODEL_DIR = MODEL_DIR_CACHED
elif MODE == "direct":
    from vgg_network import VGG_Network
    from estimator_svdd_naive import _LoadPreTrainedWeightsVGG
    from data_utils import run_dataset_through_network
    
    net = VGG_Network(include_FC_head=False)
    
    def get_train_dataset(net, reuse=False):
        dataset = train_input_fn(CLASS_NBR, TARGET_W).batch(BATCH_SIZE)
        dataset = run_dataset_through_network(dataset, net, reuse=reuse)
        return dataset.repeat()
    
    def get_test_dataset(net, reuse=False):
        dataset = test_input_fn(CLASS_NBR, TARGET_W).batch(BATCH_SIZE)
        dataset = dataset.map(lambda img, label: img)
        dataset = run_dataset_through_network(dataset, net, reuse=reuse)
        return dataset
    
    input_fn_train = lambda: get_train_dataset(net, reuse=False)
    input_fn_test = lambda: get_test_dataset(net, reuse=False)
    train_hooks = [_LoadPreTrainedWeightsVGG(net)]
    test_hooks = train_hooks
    MODEL_DIR = MODEL_DIR_DIRECT
else:
    raise Exception("MODE unknown")
    
classifier = SVDDClassifier(
    c=C,
    kernel=KERNEL,
    learning_rate=LR,
    model_dir=MODEL_DIR,
)

In [ ]:
classifier.train(
    input_fn=input_fn_train,
    steps=TRAIN_STEPS,
    hooks=train_hooks
)

# Predicting

In [ ]:
predictions = classifier.predict(
    input_fn=input_fn_test,
    hooks=test_hooks
)

predictions_list = list(predictions)
predicted_scores = np.asarray(list(map(lambda p: p["predicted_scores"], predictions_list))).astype(np.int32)
predicted_classes = np.asarray(list(map(lambda p: p["predicted_classes"], predictions_list))).astype(np.int32)

In [ ]:
df = pd.DataFrame(predicted_classes)
df.plot.hist()

# Validation and metrics

In [ ]:
# Test set
y_test = []
input_fn = test_cnn_input_fn(CLASS_NBR, CNN_OUTPUT_DIR).batch(1)
input_fn = input_fn.make_one_shot_iterator().get_next()
sess = tf.Session()
while True:
    try:
        data = sess.run(input_fn)
        y_test.append(data[1][0])
    except tf.errors.OutOfRangeError:
        break
y_test = np.asarray(y_test)
y_test.shape

In [ ]:
from validation import evaluation_summary
evaluation_summary(y_test, predicted_classes, plot_cm=True)